In [ ]:
 pip install requests pytz azure-servicebus

StatementMeta(, , -1, Waiting, , Waiting)

In [ ]:
import requests
import json

try:
    # Fetch real-time data from the Weather API
    response = requests.get("enter your api request key")
    response.raise_for_status()  # Raises an error if the request fails

    # Convert the response to JSON format
    data = response.json()

    # Print the sample API response for verification
    #print("Sample API Response:", json.dumps(data, indent=4))

    # Function to Flatten JSON and Extract Keys with Values
    def flatten_json(nested_json, parent_key=''):
        flattened = {}
        for key, value in nested_json.items():
            new_key = f"{parent_key}.{key}" if parent_key else key  # Append parent key for nested items
            if isinstance(value, dict):
                flattened.update(flatten_json(value, new_key))
            else:
                flattened[new_key] = value
        return flattened

    # Extract Column Headers with Values
    flattened_data = flatten_json(data)
    data=flattened_data

    # Print Columns with Values
    print("\nExtracted Data (Column Name : Value):\n")
    for key, value in flattened_data.items():
        print(f"{key} : {value}")

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")


StatementMeta(, , -1, Waiting, , Waiting)

In [ ]:
from datetime import datetime
import pytz

# Get the current time in Sydney, Australia
sydney_tz = pytz.timezone("Europe/London")
now_sydney = datetime.now(sydney_tz)

# Format the timestamp in different ways
formatted_datetime = now_sydney.strftime("%m/%d/%Y %I:%M:%S %p")  # MM/DD/YYYY HH:MM:SS AM/PM
date_column = now_sydney.strftime("%d-%m-%Y")  # DD-MM-YYYY
time_column = now_sydney.strftime("%H:%M:%S")  # HH:MM:SS

# Check if data is a dictionary (single record) or a list (multiple records)
if isinstance(data, dict):
    # If it's a single object, update it directly
    data["datetime"] = formatted_datetime
    data["date"] = date_column
    data["time"] = time_column
    updated_data = [data]  # Convert to a list for consistent processing
else:
    # If it's a list, iterate over each record
    for record in data:
        record["datetime"] = formatted_datetime
        record["date"] = date_column
        record["time"] = time_column
    updated_data = data

# Print the updated data structure
print("Sample Data with Timestamps:")
print(updated_data[:5]) 

StatementMeta(, , -1, Waiting, , Waiting)

In [ ]:
from azure.servicebus import ServiceBusClient, ServiceBusMessage
import json

# Replace with your Fabric EventStream connection string
myconnectionstring = "eventstream connection string"

# Function to send messages to Fabric EventStream
def send_to_eventstream(messages, connection_string):
    # Extract EntityPath from connection string (always included)
    entity_path = None
    for param in connection_string.split(';'):
        if param.startswith('EntityPath='):
            entity_path = param.split('=')[1]
            break

    # Ensure EntityPath is found (it should always be included)
    if not entity_path:
        raise ValueError("EntityPath is missing in the connection string. Please check your Fabric setup.")

    # Ensure data is always a list before sending
    if isinstance(messages, dict):
        messages = [messages]  # Convert single object to a list

    # Establish connection to Fabric EventStream
    servicebus_client = ServiceBusClient.from_connection_string(connection_string)
    try:
        with servicebus_client.get_queue_sender(entity_path) as sender:
            # Convert messages to JSON format
            batch_message = [ServiceBusMessage(json.dumps(msg)) for msg in messages]
            sender.send_messages(batch_message)
            print(f"Successfully sent {len(messages)} records to EventStream.")
    except Exception as e:
        print(f"Error sending messages: {e}")
    finally:
        servicebus_client.close()

send_to_eventstream(data, myconnectionstring)




StatementMeta(, , -1, Waiting, , Waiting)